## Initialize

In [ ]:
getwd()

In [ ]:
#library(Rmisc)
library(tidyverse)
library(glue)
library(arrow)
library(patchwork)

In [ ]:
if (grepl("sc", Sys.info()[["nodename"]], fixed=TRUE)) {
    base_path = "/sc-projects/sc-proj-ukb-cvd"
} else {
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"}
print(base_path)

dataset_name = "210714_metabolomics"
path = "/data/analysis/ag-reils/steinfej/code/umbrella/pre/ukbb"
data_path = glue("{base_path}/data")
dataset_path = glue("{data_path}/3_datasets_post/{dataset_name}")

project_label="21_metabolomics_multitask"
project_path = glue("{base_path}/results/projects/{project_label}")
figures_path = glue("{project_path}/figures")
data_results_path = glue("{project_path}/data")

## Load data

In [ ]:
list.dirs(path = project_path, full.names = TRUE, recursive = TRUE)

In [ ]:
run = "220126"

In [ ]:
DSM = "MultiTaskSurvivalTraining"
data = arrow::read_feather(glue("{dataset_path}/data_merged.feather")) 
data_description = arrow::read_feather(glue("{dataset_path}/description_merged.feather"))
predictions = arrow::read_feather(glue("{data_results_path}/predictions_{run}_metabolomics.feather")) 

In [ ]:
data_events = data %>% select(eid, ends_with("event"), ends_with("event_time")) %>% 
    pivot_longer(-eid, names_to=c("endpoint", "type"), values_to="value", names_pattern="(.*)(event_time|event)") %>% 
    mutate(endpoint = stringr::str_sub(endpoint, end=-2)) %>% pivot_wider(names_from="type", values_from="value")

In [ ]:
predictions_tte = predictions %>% left_join(data_events, by=c("endpoint", "eid"))

In [ ]:
library(ggthemes)
endpoint_map = c(
    'M_MACE'='MACE',
    'M_all_cause_dementia'='Dementia',
    'M_type_2_diabetes'='T2 Diabetes',
    'M_liver_disease'='Liver Disease',
    'M_renal_disease'='Renal Disease',
    'M_atrial_fibrillation'='Atrial Fibrillation',
    'M_heart_failure'= 'Heart Failure',
    'M_coronary_heart_disease'='CHD',
    'M_venous_thrombosis'='Ven. Thrombosis',
    'M_cerebral_stroke'='Cerebral Stroke',
    'M_abdominal_aortic_aneurysm'='AAA',
    'M_peripheral_arterial_disease'='PAD',
    "M_chronic_obstructuve_pulmonary_disease" = "COPD",
    "M_asthma" = "Asthma",
    'M_parkinsons_disease' = "Parkinson's",    
    "M_lung_cancer" = "Lung Cancer",
    "M_non_melanoma_skin_cancer" = "Skin Cancer",
    "M_colon_cancer"= "Colon Cancer",
    "M_rectal_cancer" = "Rectal Cancer",
    "M_prostate_cancer"= "Prostate Cancer",
    "M_breast_cancer" = "Breast Cancer",
    'M_cataracts' = "Cataracts", 
    'M_glaucoma' = "Glaucoma",
    'M_fractures' = "Fractures"
)

endpoint_order = c("M_type_2_diabetes", "M_all_cause_dementia", "M_heart_failure")

In [ ]:
library("jsonlite")
colors_path = "colors.json"
colors_dict = read_json(colors_path)

color_map <- c('COX_Age+Sex' = colors_dict$pastel$grey$light, 
               'DS_Metabolomics' = "#4F8EC1",  #colors_dict$pastel$orange$dark,
               'COX_ASCVD' = colors_dict$pastel$grey$mid, 
               'COX_PANEL' = colors_dict$pastel$grey$dark, 
               'DS_Age+Sex+Metabolomics' = "#53dd6c",#colors_dict$pastel$orange$mid,
               'DS_ASCVD+Metabolomics' = "#d8315b",#colors_dict$pastel$red$mid,
               'DS_PANELnoblood+Metabolomics' = "forestgreen",
               'DS_PANEL+Metabolomics' = "#1e1b18" #colors_dict$pastel$red$dark
      )

type_map <- c('COX_Age+Sex' = "dashed", 
               'DS_Metabolomics' = "solid",  #colors_dict$pastel$orange$dark,
               'COX_ASCVD' = "dashed", 
               'COX_PANEL' = "dashed", 
               'DS_Age+Sex+Metabolomics' = "solid",#colors_dict$pastel$orange$mid,
               'DS_ASCVD+Metabolomics' = "solid",#colors_dict$pastel$red$mid,
               'DS_PANEL+Metabolomics' = "solid", #colors_dict$pastel$red$dark
                'DS_PANELnoblood+Metabolomics' = "solid",
              "Treat All" = "solid",
              "Treat None" = "solid"
      )

size_map <- c('COX_Age+Sex' = 0.25, 
               'DS_Metabolomics' = 0.25,  #colors_dict$pastel$orange$dark,
               'COX_ASCVD' = 0.25, 
               'COX_PANEL' = 0.25, 
               'DS_Age+Sex+Metabolomics' = 0.25,#colors_dict$pastel$orange$mid,
               'DS_ASCVD+Metabolomics' = 0.25,#colors_dict$pastel$red$mid,
               'DS_PANEL+Metabolomics' = 0.25, #colors_dict$pastel$red$dark
                'DS_PANELnoblood+Metabolomics' = 0.25,
              "Treat All" = 0.25,
              "Treat None" = 0.25
      )

In [ ]:
library(foreach)

In [ ]:
library(doParallel)

In [ ]:
scores_dca = c("DS_Metabolomics",
    "COX_Age+Sex",
    "DS_Age+Sex+Metabolomics" ,
    "COX_ASCVD",
    "DS_ASCVD+Metabolomics",
    "COX_PANEL",
    "DS_PANEL+Metabolomics",
     "DS_PANELjustbloodcount+Metabolomics",          
    "DS_PANELnoblood+Metabolomics"  
              )

In [ ]:
calculate_dca = function(endpoint){
    dca_temp = predictions_tte %>% filter(endpoint==!!endpoint) %>% 
        mutate(score=glue("{module}_{features}")) %>% 
        select(eid, score, Ft_10, event, event_time) %>% 
        filter(score %in% scores_dca) %>%
        pivot_wider(names_from="score", values_from="Ft_10") %>% 
        distinct()
    
    print(glue("{endpoint} n={nrow(dca_temp)}"))
    flush.console()
    
    dca_endpoint = dca(
        Surv(event_time, event) ~ DS_Metabolomics + `COX_Age+Sex` + `DS_Age+Sex+Metabolomics` + COX_ASCVD + `DS_ASCVD+Metabolomics` + COX_PANEL + `DS_PANEL+Metabolomics` + `DS_PANELjustbloodcount+Metabolomics` + `DS_PANELnoblood+Metabolomics`, 
        data = dca_temp,
        time = 10,
        thresholds = seq(0, 0.5, by=0.001)
           )$dca
    return(dca_endpoint %>% mutate(st_net_benefit = net_benefit/prevalence))
}

In [ ]:
registerDoParallel(length(endpoint_order))

In [ ]:
library(dcurves)
dca_data = foreach(key=endpoint_order, endpoint=endpoint_order) %dopar% {
    calculate_dca(endpoint)
}

In [ ]:
names(dca_data) = endpoint_order
for (endpoint in endpoint_order){
    dca_data[[endpoint]]["endpoint"] = endpoint
    }

In [ ]:
dca_data_agg = bind_rows(dca_data) 

In [ ]:
base_size = 8
title_size = 10
facet_size = 10
geom_text_size=3
library(ggplot2); 
theme_set(theme_classic(base_size = base_size) + 
          theme(strip.background = element_blank(), plot.title=element_text(size=title_size, hjust=0), 
                strip.text.x = element_text(size = facet_size),axis.title=element_text(size=10), axis.text=element_text(size=8, color="black"),
                legend.position="bottom", axis.line = element_line(size = 0.2), axis.ticks=element_line(size=0.2), panel.grid.major.y = element_line()))

## AgeSex + Metabolomics vs PANEL

In [ ]:
endpoints_plot = c(
         "M_type_2_diabetes", #s
                  "M_all_cause_dementia", #s
                  "M_heart_failure" #s

scores_plot = c(
    "Treat All", 
    "Treat None",
    "COX_Age+Sex",
    "DS_Age+Sex+Metabolomics",
    "DS_PANELnoblood+Metabolomics",
    "COX_ASCVD",
    "COX_PANEL",
    "DS_PANEL+Metabolomics"
)

In [ ]:
calibration_df = predictions_tte %>% 
    filter(endpoint %in% endpoints_plot) %>% 
    mutate(score = glue("{module}_{features}")) %>% 
    filter(score %in% scores_plot) %>% 
    select(eid, endpoint, score, Ft_10, event, event_time) %>%
    group_by(endpoint, score) %>% mutate(risk_group = ntile(Ft_10, 10), 
                                         event_10 = case_when(event == 0 ~ 0,
                                                              ((event==1)&(event_time>10)) ~ 0,
                                                              ((event==1)&(event_time<=10)) ~ 1
                                                             )
                                        ) %>% 
    group_by(endpoint, score, risk_group) %>% 
    mutate(mean_risk = mean(Ft_10), mean_events = mean(event_10)) %>%
    mutate(endpoint = factor(endpoint, levels=endpoints_plot))

In [ ]:
library(lemon)

In [ ]:
plots_cal = c()
for (endpoint in endpoints_plot){
    temp = calibration_df %>% filter(as.character(endpoint) == !!endpoint)
    endpoint_label = endpoint_map[[endpoint]]
    plots_cal[[endpoint]] = 
        ggplot(temp, aes(x=mean_risk*100, y=mean_events*100, color=score, size=score, linetype=score)) + 
        geom_point() + 
        geom_line()+
        labs(title=endpoint_label, x="Predicted Risk [%]", y="Observed Event Rate [%]") +
        geom_abline(intercept=0, slope=1, color="red", alpha=0.7)+    
        scale_color_manual(values=color_map)+
        scale_linetype_manual(values=type_map)+
        scale_size_manual(values=size_map)+
        coord_cartesian(xlim=c(0, NA, ylim=c(0, NA))+
        theme(legend.position="none", axis.title = element_text())
    #print(plotsA[[endpoint]])
    flush.console()
    }

In [ ]:
fig5_cal = wrap_plots(plots_cal, ncol=3)
plot_width=8.25; plot_height=3.25; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)
fig5_cal

In [ ]:
scores_plot = c(
    "Treat All", 
    "Treat None",
    "COX_Age+Sex",
    "DS_Age+Sex+Metabolomics",
    "DS_PANELnoblood+Metabolomics",
    "COX_ASCVD",
    "COX_PANEL"
)

plotsA = c()
for (endpoint in endpoints_plot){
    temp = dca_data[[endpoint]] %>% mutate(st_net_benefit = st_net_benefit*100) %>% mutate(st_net_benefit = case_when((label == "COX_Age+Sex")&is.na(st_net_benefit) ~ 0, TRUE ~ st_net_benefit))
    if (min((temp %>% filter(label=="DS_PANEL+Metabolomics"))$st_net_benefit, na.rm=TRUE)>0){
        threshold_max = (temp %>% filter(label=="DS_PANEL+Metabolomics") %>% slice_min(st_net_benefit))$threshold
        }
    else{
        threshold_max = min((temp %>% filter(label=="DS_PANEL+Metabolomics") %>% filter(st_net_benefit<0))$threshold, na.rm=TRUE)
        }
    endpoint_label = endpoint_map[[endpoint]]
        temp_plot = temp %>% filter(label %in% scores_plot)
    temp_plot_wide = temp_plot %>%  select(threshold, label, st_net_benefit) %>% pivot_wider(names_from="label", values_from="st_net_benefit")
    plotsA[[endpoint]] = 
        ggplot(temp_plot)+
          labs(title=endpoint_label, x="Threshold Probability [%]", y="Standardized Net Benefit [%]")+
          geom_line(data=temp_plot, mapping=aes(x=threshold, y=st_net_benefit, color=label, linetype=label, size=label)) + 
        geom_ribbon(data=temp_plot_wide, mapping=aes(x=threshold, ymin=`COX_Age+Sex`, ymax=`DS_Age+Sex+Metabolomics`), fill=colors_dict$pastel$green$mid, alpha=0.1, colour = NA)+
        geom_ribbon(data=temp_plot_wide, mapping=aes(x=threshold, ymin=`DS_Age+Sex+Metabolomics`, ymax=`DS_PANELnoblood+Metabolomics`), fill=colors_dict$pastel$green$dark, alpha=0.2, colour = NA)+
          coord_cartesian(xlim=c(0, threshold_max), ylim=c(0, max(temp$st_net_benefit, na.rm=TRUE)))+
          scale_x_continuous(labels = function(x){x*100})+#labels = scales::percent_format(accuracy=1))+
          scale_color_manual(values=color_map)+
          scale_linetype_manual(values=type_map)+
    scale_size_manual(values=size_map)+
        theme(legend.position="none")
    #print(plotsA[[endpoint]])
    flush.console()
    }

In [ ]:
scores_plot = c(
    "Treat All", 
    "Treat None",
      "COX_Age+Sex",
      "COX_ASCVD",
    "COX_PANEL",
    "DS_PANEL+Metabolomics"
)

plotsB = c()
for (endpoint in endpoints_plot){
    temp = dca_data[[endpoint]] %>% mutate(st_net_benefit = st_net_benefit*100) 
    if (min((temp %>% filter(label=="DS_PANEL+Metabolomics"))$st_net_benefit, na.rm=TRUE)>0){
        threshold_max = (temp %>% filter(label=="DS_PANEL+Metabolomics") %>% slice_min(st_net_benefit))$threshold
        }
    else{
        threshold_max = min((temp %>% filter(label=="DS_PANEL+Metabolomics") %>% filter(st_net_benefit<0))$threshold, na.rm=TRUE)
        }
    endpoint_label = endpoint_map[[endpoint]]
        temp_plot = temp %>% filter(label %in% scores_plot)
    temp_plot_wide = temp_plot %>%  select(threshold, label, st_net_benefit) %>% pivot_wider(names_from="label", values_from="st_net_benefit")
    plotsB[[endpoint]] = 
        ggplot(data=temp_plot) + 
             labs(title=endpoint_label, x="Threshold Probability [%]", y="Standardized Net Benefit [%]")+
            geom_line(data=temp_plot, mapping=aes(x=threshold, y=st_net_benefit, color=label, linetype=label, size=label)) + 
            geom_ribbon(data=temp_plot_wide, mapping=aes(x=threshold, ymin=COX_PANEL, ymax=`DS_PANEL+Metabolomics`), fill="black", alpha=0.1, colour = NA)+
            coord_cartesian(xlim=c(0, threshold_max), ylim=c(0, max(temp$st_net_benefit, na.rm=TRUE)))+
            scale_x_continuous(labels = function(x){x*100})+#scales::percent_format(accuracy=1))+
            scale_color_manual(values=color_map)+
            scale_linetype_manual(values=type_map)+
            scale_size_manual(values=size_map)+
            theme(legend.position="none")
    #print(plotsB[[endpoint]])
    flush.console()
    }

In [ ]:
base_size = 8
title_size = 10
facet_size = 10
geom_text_size=3
library(ggplot2); 
theme_set(theme_classic(base_size = base_size) + 
          theme(strip.background = element_blank(), plot.title=element_text(size=title_size, hjust=0), 
                strip.text.x = element_text(size = facet_size),axis.title=element_text(size=10), axis.text=element_text(size=8, color="black"),
                legend.position="bottom", axis.line = element_line(size = 0.2), axis.ticks=element_line(size=0.2), panel.grid.major.y = element_line()))

In [ ]:
zip <- function(...) {
  mapply(list, ..., SIMPLIFY = TRUE)
}
plots = c(plotsA, plotsB)
fig5_dca = wrap_plots(plots, ncol=3)
plot_width=8.25; plot_height=6.5; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)
fig5_dca

In [ ]:
fig5 = (fig5_cal / fig5_dca) +  plot_layout(height = c(1, 2.5)) & theme(plot.title = element_text(hjust = 0.5))
plot_width=8.25; plot_height=9; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)
fig5

In [ ]:
library(gt)
plot_name = "Figures_5_ABC_ClinicalUtility"
fig5 %>% ggsave(filename=glue("outputs/{plot_name}.pdf"), device="pdf", width=plot_width, height=plot_height, dpi=plot_dpi)

In [ ]:
library(gt)
plot_name = "Figures_5_ABC_ClinicalUtility"
fig5 %>% ggsave(filename=glue("outputs/{plot_name}.png"), device="png", width=plot_width, height=plot_height, dpi=plot_dpi)